<a href="https://colab.research.google.com/github/Bitdribble/LDL/blob/main/colab/c6e1_boston.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
The MIT License (MIT)
Copyright (c) 2021 NVIDIA
Permission is hereby granted, free of charge, to any person obtaining a copy of
this software and associated documentation files (the "Software"), to deal in
the Software without restriction, including without limitation the rights to
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
the Software, and to permit persons to whom the Software is furnished to do so,
subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""


In [1]:
# Set up sandbox environment
!rm -rf LDL
!git clone https://github.com/Bitdribble/LDL.git

# Install module dependencies
%cd /content/LDL
%pip install -r colab_requirements.txt

# Download data
#!ls data/mnist
#!./data/mnist/download_mnist.sh

# cd to desired directory
%cd /content/LDL/pt_framework

Cloning into 'LDL'...
remote: Enumerating objects: 209, done.
remote: Counting objects: 100% (209/209), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 209 (delta 111), reused 169 (delta 77), pack-reused 0
Receiving objects: 100% (209/209), 1.21 MiB | 10.60 MiB/s, done.
Resolving deltas: 100% (111/111), done.
/content/LDL
  Created wheel for idx2numpy: filename=idx2numpy-1.2.3-py3-none-any.whl size=7919 sha256=dacc68cf5fbd5137bb57b34038674b5821dec7bac4c3236b6570cbaa2974acec
  Stored in directory: /root/.cache/pip/wheels/1a/ce/ad/d5e95a35cfe34149aade5e500f2edd535c0566d79e9a8e1d8a
Successfully built idx2numpy
/content/LDL/pt_framework


This code example demonstrates how to use a neural network to solve a regression problem, using the Boston housing dataset. More context for this code example can be found in the section "Programming Example: Predicting House Prices with a DNN" in Chapter 6 in the book Learning Deep Learning by Magnus Ekman (ISBN: 9780137470358).


Unlike MNIST, the Boston Housing dataset is not included with PyTorch, so we retrieve it using scikit-learn instead. This is done by calling the load_boston() function. We then retrieve the inputs and targets as NumPy arrays by calling the get() method. We explicitly split them up into a training set and a test set using the scikit-learn function train_test_split().

We convert the NumPy arrays to np.float32 and reshape them to ensure that the datatype and dimensions later match what PyTorch expects. 

We standardize both the training and test data by using the mean and standard deviation from the training data. The parameter axis=0 ensures that we compute the mean and standard deviation for each input variable separately. The resulting mean (and standard deviation) is a vector of means instead of a single value. That is, the standardized value of the nitric oxides concentration is not affected by the values of the per capita crime rate or any of the other variables.

Finally we create Dataset objects. To do that we need to first convert the NumPy arrays to PyTorch tensors. That is done by calling torch.from_numpy().


In [2]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from utilities import train_model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EPOCHS = 500
BATCH_SIZE = 16

# Read and standardize the data.
boston_housing = load_boston()
data = boston_housing.get('data')
target = boston_housing.get('target')

raw_x_train, raw_x_test, y_train, y_test = train_test_split(
    data, target, test_size=0.2, random_state=0)

# Convert to same precision as model.
raw_x_train = raw_x_train.astype(np.float32)
raw_x_test = raw_x_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)
y_train = np.reshape(y_train, (-1, 1))
y_test = np.reshape(y_test, (-1, 1))

x_mean = np.mean(raw_x_train, axis=0)
x_stddev = np.std(raw_x_train, axis=0)
x_train = (raw_x_train - x_mean) / x_stddev
x_test = (raw_x_test - x_mean) / x_stddev

# Create Dataset objects.
trainset = TensorDataset(torch.from_numpy(x_train),
                         torch.from_numpy(y_train))
testset = TensorDataset(torch.from_numpy(x_test),
                        torch.from_numpy(y_test))


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

We then create the model. The code looks follows the same pattern as c5e1_mnist_learning. We define our network to have two hidden layers, so we are now officially doing DL! The two hidden layers in our network implementation have 64 ReLU neurons each, where the first layer is declared to have 13 inputs to match the dataset. The output layer consists of a single neuron with a linear activation function. We use MSE as the loss function and use the Adam optimizer.

Instead of implementing the training loop below, we have broken it out into a separate function train_model(). Its implementation can be found in the file utilities.py. It is very similar to the training loop in c5e1_mnist_learning but has some additional logic to be able to handle both classification and regression problems. In particular, it takes a parameter "metric". If we work on a classification problem it should be set to "acc" and the function will compute accuracy. If we work on a regression problem it should be set to "mae" and the function will compute mean absolute error instead. 


In [3]:
# Create model.
model = nn.Sequential(
    nn.Linear(13, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 1)
)

# Initialize weights.
for module in model.modules():
    if isinstance(module, nn.Linear):
        nn.init.xavier_uniform_(module.weight)
        nn.init.constant_(module.bias, 0.0)

# Loss function and optimizer
optimizer = torch.optim.Adam(model.parameters())
loss_function = nn.MSELoss()

# Train model.
train_model(model, device, EPOCHS, BATCH_SIZE, trainset, testset,
            optimizer, loss_function, 'mae')


Epoch 1/500 loss: 545.1769 - mae: 20.7512 - val_loss: 454.2590 - val_mae: 17.7616
Epoch 2/500 loss: 408.9080 - mae: 17.3976 - val_loss: 286.4925 - val_mae: 13.4286
Epoch 3/500 loss: 200.3347 - mae: 11.6793 - val_loss: 110.9084 - val_mae: 7.7008
Epoch 4/500 loss: 62.4472 - mae: 6.1432 - val_loss: 66.5486 - val_mae: 5.5533
Epoch 5/500 loss: 37.6736 - mae: 4.4545 - val_loss: 50.4528 - val_mae: 4.9521
Epoch 6/500 loss: 27.3155 - mae: 3.7503 - val_loss: 44.2256 - val_mae: 4.6478
Epoch 7/500 loss: 25.4908 - mae: 3.5196 - val_loss: 40.1497 - val_mae: 4.4042
Epoch 8/500 loss: 21.4497 - mae: 3.3100 - val_loss: 38.3948 - val_mae: 4.2460
Epoch 9/500 loss: 19.5296 - mae: 3.1965 - val_loss: 35.3808 - val_mae: 4.0514
Epoch 10/500 loss: 18.5765 - mae: 3.0794 - val_loss: 32.4696 - val_mae: 3.8915
Epoch 11/500 loss: 16.6848 - mae: 2.9574 - val_loss: 31.2790 - val_mae: 3.7680
Epoch 12/500 loss: 15.9561 - mae: 2.8176 - val_loss: 29.0453 - val_mae: 3.6298
Epoch 13/500 loss: 14.8895 - mae: 2.7690 - val_los

[0.6176272389980463, 2.8120128767830983]

After the training is done, we use our model to predict the price for all test examples and then print out the first four predictions and the correct values so we can get an idea of how correct the model is.


In [4]:
# Print first 4 predictions.
inputs = torch.from_numpy(x_test)
inputs = inputs.to(device)
outputs = model(inputs)
for i in range(0, 4):
    print('Prediction: %4.2f' % outputs.data[i].item(),
         ', true value: %4.2f' % y_test[i].item())


Prediction: 22.92 , true value: 22.60
Prediction: 29.77 , true value: 50.00
Prediction: 23.79 , true value: 23.00
Prediction: 11.60 , true value: 8.30
